In [1]:
%load_ext tensorboard
import tensorflow as tf
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataset import random_split
from torchvision.transforms import ToTensor, Lambda

import numpy as np
from scipy.io import wavfile
from scipy.fft import fft, fftfreq
from scipy.fftpack import dct
import IPython.display
import librosa

import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from matplotlib import cm
%matplotlib inline
import pandas as pd;
import numpy as np
import math
from PIL import Image
from IPython.display import display

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

import mne
import mne.channels

# Model


In [3]:
class AbnormalNeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        # Input 5 Channels
        self.CNN1 = nn.Conv1d(in_channels=1, out_channels=17, kernel_size=10)
        self.MaxPool1 = nn.MaxPool1d(kernel_size=2, stride=2)

        self.CNN2 = nn.Conv1d(in_channels=17, out_channels=29, kernel_size=10)
        self.MaxPool2 = nn.MaxPool1d(kernel_size=2, stride=2)

        self.CNN3 = nn.Conv1d(in_channels=29, out_channels=52, kernel_size=2)
        self.MaxPool3 = nn.MaxPool1d(kernel_size=2, stride=1)

        # self.flatten = nn.Flatten()

        self.fc1 = nn.Linear(1, 100)
        self.dropout1 = nn.Dropout(p=0.3)
        # RuntimeError: mat1 and mat2 shapes cannot be multiplied (200x1 and 200x100)

        self.fc2 = nn.Linear(100, 50)
        self.dropout2 = nn.Dropout(p=0.3)

        # Output Layer
        self.output = nn.Linear(in_features=50, out_features=1)

    def forward(self, x):
        x = F.relu(self.CNN1(x))
        x = self.MaxPool1(x)

        x = F.relu(self.CNN2(x))
        x = self.MaxPool2(x)

        x = F.relu(self.CNN3(x))
        x = self.MaxPool3(x)

        # x = self.flatten(x)

        x = F.relu(self.fc1(x))
        x = self.dropout1(x)

        x = F.relu(self.fc2(x))
        x = self.dropout2(x)

        x = self.output(x)
        return x


# Set device to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize model
model = AbnormalNeuralNetwork().to(device)

# Set optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.MSELoss()

# Calculate number of parameters in model
n_params = 0
for x in model.parameters():
    n_params += len(torch.flatten(x))
print(f"Number of parameters in model: {n_params:,}")

# Print Model
model = model.to(device)

Number of parameters in model: 13,515


# Load


In [5]:
model.load_state_dict(torch.load("model_first_test"))
model.eval()

AbnormalNeuralNetwork(
  (CNN1): Conv1d(1, 17, kernel_size=(10,), stride=(1,))
  (MaxPool1): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (CNN2): Conv1d(17, 29, kernel_size=(10,), stride=(1,))
  (MaxPool2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (CNN3): Conv1d(29, 52, kernel_size=(2,), stride=(1,))
  (MaxPool3): MaxPool1d(kernel_size=2, stride=1, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=1, out_features=100, bias=True)
  (dropout1): Dropout(p=0.3, inplace=False)
  (fc2): Linear(in_features=100, out_features=50, bias=True)
  (dropout2): Dropout(p=0.3, inplace=False)
  (output): Linear(in_features=50, out_features=1, bias=True)
)